# **Chapter 11: Advanced Functions & Patterns**
Welcome! This chapter turns you from a Python user into a **Python power user**.  
You’ll learn techniques that make code **cleaner, faster, and easier to reuse**.

### What you'll master
1. Closures  
2. Function Decorators  
3. Class Decorators  
4. Higher-Order Functions (map/filter/reduce, composition)  
5. Advanced Comprehensions  
6. Partial Functions  
7. Caching (lru_cache)  
8. Advanced Iterators & Generators  
9. Context Managers

> **Tip:** Run cells as you read. Tweak values, re-run, and watch behavior change.


## 11.1 Closures
A **closure** is a function that remembers variables from the scope where it was created—even after that scope has finished.

**Mental model:**  
```
def outer():
    x = 10        # <- captured
    def inner():  # <- closure
        return x
    return inner
```
The returned `inner` function **remembers** `x`.

### Why closures?
- Keep **state** without classes (counters, throttles, feature flags)
- Create **configurable functions** (validators, formatters)
- Light-weight **encapsulation** (hide details)


### Example: Stateful Counter

In [ ]:
def make_counter(start=0):
    count = start
    def inc(step=1):
        nonlocal count
        count += step
        return count
    return inc

c1 = make_counter(10)
c2 = make_counter(100)

print("c1:", c1())       # 11
print("c1:", c1(5))      # 16
print("c2:", c2())       # 101


**Visualization**
```
make_counter(10)
|
|-- count = 10
|-- returns inc() -> remembers count
```
Closures are like giving a function **memory** of its environment.

### Example: Validator Factory

In [ ]:
def length_validator(min_len=0, max_len=10):
    def validate(s: str) -> bool:
        return min_len <= len(s) <= max_len
    return validate

name_ok = length_validator(3, 8)
print(name_ok("Ali"))        # True
print(name_ok("A"))          # False
print(name_ok("Alexander"))  # False


### Exercises (Closures)
1) Write `discount(percent)` that returns a function `apply(price)` → returns price after discount.  
2) Implement `call_counter()` that returns a function which increments and returns how many times it has been called.  
3) Make `remember_last_k(k)` that returns a function; every call saves the input and returns the last *k* inputs.


## 11.2 Function Decorators
A **decorator** is a callable that takes a function and returns a new function (often adding behavior).

**Analogy:** Gift 🎁 wrapping — same gift, extra presentation (logging, timing, checks).

### Basic decorator

In [ ]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Before")
        result = func(*args, **kwargs)
        print("After")
        return result
    return wrapper

@my_decorator
def greet(name):
    print(f"Hello, {name}!")

greet("Aisha")


### Logging decorator (with metadata preserved via `functools.wraps`)

In [ ]:
import functools

def logger(fn):
    @functools.wraps(fn)
    def wrapper(*args, **kwargs):
        print(f"[LOG] {fn.__name__} args={args} kwargs={kwargs}")
        return fn(*args, **kwargs)
    return wrapper

@logger
def add(a, b):
    return a + b

print("sum:", add(5, 3))
print("preserved name:", add.__name__)


### Timer decorator + stacking
Decorators can be stacked like layers of a cake 🎂.

In [ ]:
import time, functools

def timer(fn):
    @functools.wraps(fn)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        try:
            return fn(*args, **kwargs)
        finally:
            print(f"[TIMER] {fn.__name__} took {time.perf_counter() - start:.6f}s")
    return wrapper

@logger
@timer
def work(n=20000):
    s = 0
    for i in range(n):
        s += i*i
    return s

print("work result:", work(50000))


### Decorator with parameters
3-layer design: `decorator_factory(args) -> decorator(fn) -> wrapper(...)`.


In [ ]:
import functools

def require_role(role):
    def decorator(fn):
        @functools.wraps(fn)
        def wrapper(user, *args, **kwargs):
            if getattr(user, "role", None) != role:
                raise PermissionError("forbidden")
            return fn(user, *args, **kwargs)
        return wrapper
    return decorator

class User:
    def __init__(self, name, role):
        self.name, self.role = name, role

@require_role("admin")
def delete_record(user, rid):
    return f"deleted {rid}"

admin = User("Ali", "admin")
viewer = User("Aisha", "viewer")
print(delete_record(admin, 42))
# delete_record(viewer, 42)  # Uncomment to see the exception


### Exercises (Decorators)
1) Build `@retry(n=3)` that retries on exception (exponential backoff optional).  
2) Build `@cache_dict` that caches results in a dict (keyed by args).  
3) Create `@validate_nonempty` that rejects empty-string arguments.


## 11.3 Class Decorators
A **class decorator** takes a class and returns a modified class. Use for registration, auto-methods, or invariants.

### Auto `__repr__` and registry examples

In [ ]:
def auto_repr(cls):
    def __repr__(self):
        kv = ", ".join(f"{k}={v!r}" for k, v in self.__dict__.items())
        return f"{cls.__name__}({kv})"
    cls.__repr__ = __repr__
    return cls

@auto_repr
class Person:
    def __init__(self, name, age):
        self.name, self.age = name, age

print(Person("Sara", 21))

REGISTRY = {}

def register(cls):
    REGISTRY[cls.__name__] = cls
    return cls

@register
class Foo: pass

@register
class Bar: pass

print("REGISTRY:", REGISTRY)


### Exercises (Class Decorators)
1) Make `@enforce_attrs(required=("id","name"))` that raises in `__init__` if missing.  
2) Make `@frozen` that prevents attribute reassignment after `__init__`.


## 11.4 Higher-Order Functions (HOFs)
Functions that **take functions** or **return functions**. Great for pipelines.

### Composition + map/filter/reduce examples

In [ ]:
from functools import reduce

def compose(f, g):
    return lambda x: f(g(x))

def strip(s): return s.strip()
def lower(s): return s.lower()
def not_empty(s): return bool(s)

inputs = ["  Hello ", "  ", "World", "  PY  "]
cleaned = list(map(lower, filter(not_empty, map(strip, inputs))))
print("cleaned:", cleaned)

nums = [1,2,3,4]
sum_sq = reduce((lambda acc, x: acc + x*x), nums, 0)
print("sum of squares:", sum_sq)


### Exercises (HOFs)
1) Build `pipeline(funcs)` that takes a list of functions and returns a composed function.  
2) Using `map/filter`, normalize a list of emails: strip, lower, and keep only ones containing `'@'`.


## 11.5 Advanced Comprehensions
Concise list/dict/set construction with conditions and nesting. Use for clarity, not cleverness.

### Flatten, frequency, and set-normalization examples

In [ ]:
# Nested list comprehension: flatten matrix
matrix = [[1,2,3],[4,5,6]]
flat = [x for row in matrix for x in row]
print("flat:", flat)

# Dict comprehension: word frequency
text = "to be or not to be"
freq = {w: sum(1 for t in text.split() if t == w) for w in set(text.split())}
print("freq:", freq)

# Set comprehension: unique normalized items
items = {"  A ", " a", "B ", "b  "}
normalized = {i.strip().lower() for i in items}
print("normalized:", normalized)


### Exercises (Comprehensions)
1) Build a dict `{word: length}` for unique words in a sentence.  
2) Create a 3x3 identity matrix with a nested list comprehension.


## 11.6 Partial Functions
`functools.partial` pre-fills arguments, creating specialized callables.

### Examples: math and printing

In [ ]:
from functools import partial

def power(base, exp):
    return base ** exp

square = partial(power, exp=2)
cube = partial(power, exp=3)
print("square(5):", square(5))
print("cube(2):", cube(2))

# preconfigured printer
debug = partial(print, "[DEBUG]")
debug("Ready", 123)


### Exercises (Partial)
1) Create `kebab_join = partial(str.join, '-')` and use it on lists of words.  
2) Make `percent = partial(power, exp=1)` trivial; then change `power` to handle floats and test partials.


## 11.7 Caching with `functools.lru_cache`
Memoization speeds up repeated calls to pure functions.

### Fibonacci example (plain vs cached)

In [ ]:
import time
from functools import lru_cache

def fib_plain(n):
    return n if n < 2 else fib_plain(n-1) + fib_plain(n-2)

@lru_cache(maxsize=None)
def fib_fast(n):
    return n if n < 2 else fib_fast(n-1) + fib_fast(n-2)

start = time.time(); print("plain(25):", fib_plain(25)); print("plain time:", round(time.time()-start,3), "s")
start = time.time(); print("fast(35):", fib_fast(35));  print("fast time:", round(time.time()-start,3), "s")


### Exercises (Caching)
1) Cache a slow factorial recursion and compare timings.  
2) Add an argument to control `maxsize` and see impact.


## 11.8 Advanced Iterators & Generators
- **Iterator class**: implement `__iter__` and `__next__`  
- **Generator function**: uses `yield` for lazy sequences

### Even iterator + generator pipeline examples

In [ ]:
class Evens:
    def __init__(self, n):
        self.n = n
        self.i = 0
    def __iter__(self):
        return self
    def __next__(self):
        self.i += 2
        if self.i > self.n:
            raise StopIteration
        return self.i

print(list(Evens(10)))

def read_lines(lines):
    for line in lines:
        yield line.strip()

def to_ints(lines):
    for line in lines:
        if line:
            yield int(line)

def only_even(nums):
    for n in nums:
        if n % 2 == 0:
            yield n

sample = ["1","2","3","4",""]
print(list(only_even(to_ints(read_lines(sample)))))


### Exercises (Iterators/Generators)
1) Create a generator that yields powers of two up to a limit.  
2) Build a pipeline that reads CSV lines, parses ints, filters > threshold, and computes a sum.


## 11.9 Context Managers
Ensure resources are cleaned up automatically using `with`.

### Timer & safe file handle examples

In [ ]:
from contextlib import contextmanager
import time, os

@contextmanager
def timer(label="block"):
    start = time.perf_counter()
    try:
        yield
    finally:
        print(f"[TIMER] {label}: {time.perf_counter() - start:.6f}s")

with timer("sum squares"):
    _ = sum(i*i for i in range(100_000))

class OpenPath:
    def __init__(self, path, mode="w", encoding="utf-8"):
        self.path, self.mode, self.encoding = path, mode, encoding
        self.f = None
    def __enter__(self):
        os.makedirs(os.path.dirname(self.path), exist_ok=True)
        self.f = open(self.path, self.mode, encoding=self.encoding)
        return self.f
    def __exit__(self, exc_type, exc, tb):
        if self.f:
            self.f.close()

with OpenPath("tmp/data.txt") as f:
    f.write("hello context managers")


### Exercises (Context Managers)
1) Write a context manager that temporarily changes `os.getcwd()` (use `os.chdir`) and then restores it.  
2) Create a context manager that silences stdout for a block.


## Summary & Final Challenges
**You now have a professional toolkit:**
- Closures for state & config
- Decorators for cross-cutting concerns
- Class decorators for meta-features
- HOFs & comprehensions for pipelines
- Partial & caching for speed and ergonomics
- Iterators & generators for lazy data
- Context managers for safety

### Final Challenges
1) Build a mini authorization layer:
   - Closure to hold current user session
   - `@require_role` decorator to guard actions
   - Context manager to audit actions (write to a file)
2) Create a data pipeline:
   - Read a large text file lazily
   - Normalize & filter tokens
   - Count frequencies with comprehensions
   - Cache expensive stemming or normalization
